In [1]:
!python -m pip install --upgrade pip
!pip install matplotlib
!pip install tensorflow
!pip install pandas

In [2]:
import pandas as pd

df = pd.read_csv("Bitcoin_Trainingdata.csv")

#get percentage change in Open values
df["Open_previous"] = df["Open"].shift(1)
df["Change_in_Open"] = (df["Open"] - df["Open_previous"]) / df["Open_previous"]
df = df.dropna()

df.tail()

,Date,Open,High,Low,Close,Volume,Currency,Open_previous,Change_in_Open
2787,2022-05-05,39695.746094,39789.281250,35856.515625,36575.140625,43106256317,USD,37748.011719,0.051598
2788,2022-05-06,36573.183594,36624.359375,35482.132812,36040.921875,37795577489,USD,39695.746094,-0.078662
2789,2022-05-07,36042.503906,36129.925781,34940.824219,35501.953125,24375896406,USD,36573.183594,-0.014510
2790,2022-05-08,35502.941406,35502.941406,33878.964844,34059.265625,36763041910,USD,36042.503906,-0.014970
2791,2022-05-09,34039.796875,34210.621094,31337.292969,31425.814453,49328988160,USD,35502.941406,-0.041212


In [3]:
import numpy as np

X = []
Y = []

changes = df["Change_in_Open"]

for i in range(len(changes), 20, -1):
    Y.append(changes[i])
    X.append(np.array(changes[i-21:i-1]))
    
X = np.array(X).reshape(-1, 20, 1)
Y = np.array(Y)

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM

model = Sequential()
model.add(LSTM(1, input_shape=(20, 1)))

model.compile(optimizer="rmsprop", loss="mse")
model.fit(X, Y, batch_size=32, epochs=10)

Epoch 1/10


In [ ]:
predictions = model.predict(X).reshape(-1)
predictions = np.append(predictions, np.zeros(20))

df["predictions"] = predictions
df["Open_predicted"] = df["Open_previous"] * (1 + df["predictions"])

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook

years = mdates.YearLocator()
months = mdates.MonthLocator()
yearsFmt = mdates.DateFormatter('%Y')

#format the ticks
ax = plt.gca()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(yearsFmt)
ax.xaxis.set_minor_locator(months)

dates = np.array(df["Date"]).astype(np.datetime64)

plt.plot(df["Date"], df["Open"], label="Real")
plt.plot(df["Date"], df["Open_predicted"], label="Predicted")

plt.legend()

plt.show()

In [ ]:
df["Date"]